In [186]:
from hmm.hmm import HMM, sample_poisson_stimuli

import numpy as np

In [187]:
training_data = np.genfromtxt("../../data/Ex_2.csv", delimiter="," ,dtype=int)[1:, 1:]

In [188]:
gamma = 0.1
beta = 0.2
alpha = 0.01
rates = [1, 20]

# This is uppercase-gamma.
transition_matrix = np.array(
    [[1 - gamma, 0, gamma], [0, 1 - gamma, gamma], [beta / 2, beta / 2, 1 - beta]]
)

In [189]:
hmm = HMM(transition_matrix, alpha, lambda z: sample_poisson_stimuli(z, rates), states=[0, 1, 2], rates=rates)

In [190]:
num_nodes = 8
time_steps = 50
initial_c = 2

In [191]:
true_processing_modes, true_focus, observations = hmm.forward(
    num_nodes,
    time_steps,
    initial_c,
)

Dimensions of the joint distribution will be (T - 1, num possible Cs at t, num possible Cs at t +1)

In [192]:
joint_probabilities_normalised = hmm.infer(observations)

In [193]:
joint_probabilities_normalised.shape

(49, 3, 3)

In [221]:
num_simulations = 1000
correct_C = 0
correct_Z = 0

for _ in range(num_simulations):
    true_processing_modes, true_focus, observations = hmm.forward(num_nodes, time_steps, initial_c)
    joint_probabilities_normalised = hmm.infer(observations)
    # Compute the marginal probabilities of C at each time step
    marginal_prob_C = np.sum(joint_probabilities_normalised, axis=2)

    # Calculate the estimated C at each time step
    estimated_C = np.argmax(marginal_prob_C, axis=1)

    # Compute the most likely Z given the estimated C
    estimated_Z = np.zeros((time_steps, num_nodes), dtype=int)

    for t, c in enumerate(estimated_C):
        estimated_Z[t] = hmm.sample_hidden_z(num_nodes, c)

    # Compute the accuracy
    correct_C += np.sum(true_processing_modes[:-1] == estimated_C) / time_steps
    correct_Z += np.sum(true_focus == estimated_Z) / (time_steps * num_nodes)

In [222]:
correct_C /= num_simulations
correct_Z /= num_simulations

print(f"Proportion of correct C estimations: {correct_C:.2f}")
print(f"Proportion of correct Z estimations: {correct_Z:.2f}")

Proportion of correct C estimations: 0.58
Proportion of correct Z estimations: 0.75


> ...it may be more convenient to test the implementation in other ways. You may, for instance, observe that
>
> $$P?(Z_{t,i}=z)-P(Z_{t,i}=z|X=x)$$
>
> has mean 0 and likewise for Ct. Using simulations you can compute such quantities, with $P(Z_{t,i}=z|X=x)$ computed by the inference algorithm, and empirically check if their averages across many replications of the simulations are zero.

We then run the implementation on the real data

In [196]:
joint_probabilities_normalised = hmm.infer(training_data)
time_steps = joint_probabilities_normalised.shape[0]

In [197]:
joint_probabilities_normalised.shape

(99, 3, 3)

In [198]:
# Compute the marginal probabilities of C at each time step
marginal_prob_C = np.sum(joint_probabilities_normalised, axis=2)

# Calculate the estimated C at each time step
estimated_C = np.argmax(marginal_prob_C, axis=1)

# Compute the most likely Z given the estimated C
estimated_Z = np.zeros((time_steps, num_nodes), dtype=int)

for t, c in enumerate(estimated_C):
    estimated_Z[t] = hmm.sample_hidden_z(num_nodes, c)

In [199]:
estimated_C

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [200]:
estimated_Z

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0,

In [201]:
true_processing_modes

[2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]